<a href="https://colab.research.google.com/github/arthurpeter/Machine_Learning/blob/main/RAG_pipeline_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import xml.etree.ElementTree as ET
from pathlib import Path
from typing import List, Optional, Union

# Pentru a permite type hinting cu 'Node' în interiorul definiției clasei
from __future__ import annotations

class Node:
    """
    Reprezintă un nod într-o structură de arbore, similar cu un element XML.

    Atribute:
        label (str): Eticheta nodului (de ex., tag-ul XML).
        content (str): Conținutul text al nodului.
        children (List[Node]): O listă a nodurilor copil.
    """

    def __init__(self, label: str = "", content: str = ""):
        """Inițializează un nod nou."""
        self.label: str = label
        self.content: str = content
        self.children: List[Node] = []

    def print_node(self, level: int = 0):
        """
        Afișează structura arborelui începând de la acest nod,
        cu indentare pentru vizualizare.
        """
        indent = "  " * level
        if not self.children:
            print(f"{indent}Level {level} - Label: {self.label}, Content: {self.content}")
        else:
            print(f"{indent}Level {level} - Label: {self.label}, Content: {self.content}")
            for child in self.children:
                child.print_node(level + 1)

    @staticmethod
    def generate_tree_from_xml(xml_string: str) -> Optional[Node]:
        """
        Generează un arbore de noduri 'Node' dintr-un string XML.

        Args:
            xml_string: String-ul care conține XML-ul.

        Returns:
            Nodul rădăcină al arborelui sau None dacă parsarea eșuează.
        """
        try:
            # Parsează string-ul XML
            root_element = ET.fromstring(xml_string)
            # Începe construcția recursivă a arborelui
            return Node._build_node(root_element)
        except ET.ParseError as e:
            print(f"Eroare la parsarea XML: {e}")
            return None

    @staticmethod
    def _build_node(element: ET.Element) -> Node:
        """
        Metodă ajutătoare recursivă pentru a construi arborele
        din elemente XML (din ElementTree).
        """
        node = Node()
        # Setează eticheta (label) ca fiind tag-ul elementului XML
        node.label = element.tag

        # Setează conținutul ca fiind textul din interiorul elementului
        # .strip() curăță spațiile albe de la început și sfârșit
        if element.text:
            node.content = element.text.strip()
        else:
            node.content = "" # Potrivește logica Java (conținut gol vs. None)

        # Construiește recursiv copiii
        for child_element in element:
            node.children.append(Node._build_node(child_element))

        return node

    def get_content(self) -> str:
        """
        Returnează o reprezentare sub formă de string a nodului și
        a sub-arborelui său, recursiv.

        Format: "label: content" (pentru frunze)
        Format: "label: [child1_content]; [child2_content]" (pentru noduri cu copii)
        """
        result = []
        if not self.children:
            # Este un nod frunză (fără copii)
            content_str = self.content if self.content else ""
            result.append(f"{self.label}: {content_str}")
        else:
            # Este un nod părinte
            # Colectează recursiv conținutul de la toți copiii
            child_contents = [child.get_content() for child in self.children]
            # Unește conținutul copiilor cu "; "
            joined_content = "; ".join(child_contents)
            result.append(f"{self.label}: {joined_content}")

        return "".join(result)

    def get_split_content(self, max_length: int) -> List[str]:
        """
        Preia conținutul fiecărui copil DIRECT și îl returnează ca
        o listă, doar dacă NICIUN copil nu depășește max_length.

        Dacă un singur copil depășește limita, returnează o listă goală.
        """
        parts = []
        for child in self.children:
            child_content = child.get_content()
            if len(child_content) > max_length:
                # Un copil depășește limita, returnăm o listă goală
                # (conform logicii Java)
                return []
            parts.append(child_content)

        # Toți copiii au fost sub limită
        return parts


# --- Execuție principală (echivalentul 'public static void main') ---

if __name__ == "__main__":
    with open("response.xml", "r") as f:
      xml = f.read()


    # Exemplu de citire din fișier (dacă ai nevoie)
    # Decomentează liniile de mai jos și comentează linia de mai sus
    # pentru a citi din fișierul specificat.

    # xml_file_path_str = "C:/Users/arthurp/Documents/Work/MCP/api_wrapper/src/main/java/dvh/mcp/api_wrapper/dto/response.xml"
    # xml_file_path = Path(xml_file_path_str)
    #
    # try:
    #     # Citim conținutul fișierului
    #     xml_data_din_prompt = xml_file_path.read_text(encoding="utf-8")
    # except FileNotFoundError:
    #     print(f"EROARE: Fișierul nu a fost găsit la calea: {xml_file_path}")
    #     exit()
    # except Exception as e:
    #     print(f"EROARE la citirea fișierului: {e}")
    #     exit()


    # 1. Generează arborele din string-ul XML
    tree = Node.generate_tree_from_xml(xml)

    if tree:
        # --- O poți folosi pentru debug, ca în Java ---
        # print("--- Structura Arborelui ---")
        # tree.print_node()
        # print("-" * 30)

        # --- Poți vedea conținutul complet, recursiv ---
        # print("\n--- Conținut Complet (Recursiv) ---")
        # print(tree.get_content())
        # print("-" * 30)

        # 2. Execută logica din 'main' (getSplitContent pe nodul rădăcină)
        print("\n--- Conținut Împărțit (pe copiii rădăcinii) ---")
        # Folosim o limită foarte mare, ca în exemplul tău
        contents = tree.get_split_content(100000)

        # 3. Afișează rezultatele
        for content in contents:
            print(content, "\n\n") # Adăugăm linii goale ca în Java

        print(f"Număr total de elemente (copii ai rădăcinii): {len(contents)}")

In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch

# Load model on GPU
model = SentenceTransformer('intfloat/multilingual-e5-large', device='gpu')

# Suppose you already chunked your documents
texts = contents

# Precompute all text embeddings (do this once)
text_embeddings = model.encode(texts, convert_to_tensor=True, batch_size=32, show_progress_bar=True)

# When a question comes in:
question = "In ce data a fost primita oferta de la KRAFTEC PROFORMA?"
question_embedding = model.encode(question, convert_to_tensor=True)

# Compute cosine similarities
scores = util.cos_sim(question_embedding, text_embeddings)[0]

# Find best match
best_idx = torch.argmax(scores).item()
print(f"Best match: {texts[best_idx]} (Score: {scores[best_idx]:.3f})")
